In [ ]:
# Attack imports
from simba_test import simba_measure_dist

# Helper imports
from data import CIFAR10, IMAGENETTE
import utils

# Numerical computing and display imports
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Other imports
from datetime import datetime
import logging
import pickle
import os

In [ ]:
# Load and Prepare the Data
folder_path = '../../images/'
dataset = 'imagenette'    
if dataset == 'cifar10':
    images = CIFAR10()
if dataset == 'imagenette':
    images = IMAGENETTE()
x = images.load()
images.save_to_disk(x, folder_path, num_images=100)
folder_path = utils.move_data_to_temp_ram(folder_path, ram_size_mb=50)

In [ ]:
# SIMBA Hyperparams
simba_epsilon = 0.9
simba_hamming_threshold = 1.0                          
simba_max_steps = 20000

In [ ]:
results = list()

for i in range(3):
    # Format the path to the input image and target image
    img_path = '../../images/1.jpeg' 
    _, _, _, _, path, filetype = img_path.split('.')
    img_path = path.split('/')
    img_path = f'{folder_path}{img_path[2]}.{filetype}'

    target_img_path = f'../../images/{i+2}.jpeg' 
    _, _, _, _, path, filetype = target_img_path.split('.')
    target_img_path = path.split('/')
    target_img_path = f'{folder_path}{target_img_path[2]}.{filetype}'


    (orig_add, orig_sub), (target_add, target_sub) = simba_measure_dist(
                                    img_path=img_path, 
                                    target_path=target_img_path,
                                    eps=simba_epsilon, 
                                    mismatched_threshold=simba_hamming_threshold, 
                                    max_steps=simba_max_steps
                                    )
     
    attack_data = {
        'Image Path': img_path,
        'Target Path': target_img_path,
        'Add Dist from Orig': orig_add,
        'Add Dist to Target': target_add,
        'Sub Dist from Orig': orig_sub,
        'Sub Dist to Target': target_sub
    }
    results.append(attack_data)
    
# Save the results
file_path = 'results/orig_target_hamming_dists.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(results, file)

In [ ]:
file_path = 'results/orig_target_hamming_dists.pkl'
info = pickle.load(open(file_path, "rb"))
info[0].keys()

In [ ]:
def plot_results(info, type='Add'):
    avg_dist_from_orig = np.array([np.array(result[f'{type} Dist from Orig']) for result in info]).mean(axis=0)
    avg_dist_to_target = np.array([np.array(result[f'{type} Dist to Target']) for result in info]).mean(axis=0)
    steps = np.linspace(1, len(avg_dist_from_orig), num=len(avg_dist_from_orig))
    plt.plot(steps, avg_dist_from_orig, label='Hamming Distance from Original Hash')
    plt.plot(steps, avg_dist_to_target, label='Hamming Distance to Target Hash')
    #plt.title(f'Average Hamming Distances Relative to Original and Target {type} Image')
    plt.legend()
    plt.grid()

In [ ]:
plot_results(info, 'Add')

In [ ]:
plot_results(info, 'Sub')